In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import numpy as np
import os
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import dask.dataframe as dd
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
import random

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
import zipfile

# Replace the zip_file_path with the path to the zip file in your Google Drive
zip_file_pathElevation = '/content/drive/MyDrive/Colab Notebooks/VaesSOC/Data_Files_Redux/TensorFilesRaster/ElevationTensor_2015_Redux.zip'

zip_file_paths = [zip_file_pathElevation]

# Replace the destination_folder with the path of the folder where you want to extract the contents
destination_folderElevation = '/content/dataElevation'
destination_folders = [destination_folderElevation]

In [4]:
# Unzip the file

for i in range(len(destination_folders)):
  with zipfile.ZipFile(zip_file_paths[i], 'r') as zip_ref:
      zip_ref.extractall(destination_folders[i])


In [5]:
sampleCoordinatesElevationIDArrayPositionDf_file = '/content/drive/MyDrive/Colab Notebooks/VaesSOC/Data_Files_Redux/CoordinatesPoints/sampleCoordinatesElevationIDArrayPositionDf_Redux.parquet'
sampleCoordinatesElevationIDArrayPositionDf = dd.read_parquet(sampleCoordinatesElevationIDArrayPositionDf_file).compute()


# Part 2 Custom Dataset

In [6]:

class CustomRasterDataset(torch.utils.data.Dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, dataFrame, file_path, file_extension,windowSize,re_scale = False,new_min = -1,new_max = 1):
        'Initialization'
        self.re_scale = re_scale
        self.new_min  = new_min
        self.new_max = new_max
        self.dataFrame = dataFrame
        self.file_path = file_path
        self.file_extension = file_extension
        self.windowSize = windowSize
        self.offset = self.windowSize // 2


    def __len__(self):
        'Denotes the total number of samples'
        return len(self.dataFrame)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        ID =  self.dataFrame.iloc[index]['ID'] # str(ID).rstrip('.0')

        x = self.dataFrame.iloc[index]['x'] + random.choice([-2, -1, 0, 1, 2])
        y = self.dataFrame.iloc[index]['y'] + random.choice([-2, -1, 0, 1, 2])

        # Load data and get label
        fullArray = torch.load(self.file_path+ID+self.file_extension)
        # Determine the window for the square
        left = x - self.offset
        right = x + (self.offset + 1)
        top = y - self.offset
        bottom = y + (self.offset + 1)
        X = fullArray[left:right,top:bottom].clone().detach()  # Access value in gpu_dictElevation
        # if self.re_scale:
          # X =  re_scale(X,self.new_min,self.new_max)
        return X.unsqueeze(0)

In [7]:
file_pathElevation = '/content/dataElevation/'


In [8]:
batch_sizeElevation = 32

In [9]:
file_extension = '.pt'
num_workers = 2

In [10]:
windowSizeElevation = 65

# Create the dataset instance
datasetElevation = CustomRasterDataset(sampleCoordinatesElevationIDArrayPositionDf, file_pathElevation, file_extension,windowSizeElevation)

In [11]:

# Create a DataLoader for batching and parallel data loading (you can adjust batch_size and num_workers as needed)

dataLoaderElevation = DataLoader(datasetElevation, batch_size=batch_sizeElevation, num_workers=num_workers, shuffle=True)

# Now you can use dataLoaderEvapo in your training loop to efficiently access the elevation values in gpu_dictElevation.


In [12]:
# Assuming you have already created the 'elevation_dataloader' as mentioned in the previous steps

# Get the first batch from the dataloader using the 'next' function
first_batch = next(iter(dataLoaderElevation))
second_batch = next(iter(dataLoaderElevation))
# Print the content of the first batch
print("First Batch:")
print(first_batch.shape)


<ipython-input-6-59023ef0d4a7>:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  fullArray = torch.load(self.file_path+ID+self.file_extension)
<ipython-input-6-59023ef0d4a7>

First Batch:
torch.Size([32, 1, 65, 65])


# Part 3. Architecture of the Elevation VAE

In [13]:
import torch.optim as optim

In [14]:
def count_parameters(model: nn.Module) -> int:
    """
    Count the number of parameters in a PyTorch model.

    Args:
        model (nn.Module): PyTorch model.

    Returns:
        int: Total number of parameters.
    """
    return sum(p.numel() for p in model.parameters())


In [15]:

class ResDown(nn.Module):
    """
    Residual down sampling block for the encoder
    """

    def __init__(self, channel_in, channel_out, kernel_size=3):
        super(ResDown, self).__init__()
        self.conv1 = nn.Conv2d(channel_in, channel_out // 2, kernel_size, 2, kernel_size // 2)
        self.bn1 = nn.BatchNorm2d(channel_out // 2, eps=1e-4)
        self.conv2 = nn.Conv2d(channel_out // 2, channel_out, kernel_size, 1, kernel_size // 2)
        self.bn2 = nn.BatchNorm2d(channel_out, eps=1e-4)

        self.conv3 = nn.Conv2d(channel_in, channel_out, kernel_size, 2, kernel_size // 2)

        self.act_fnc = nn.ELU()

    def forward(self, x):
        skip = self.conv3(x)
        x = self.act_fnc(self.bn1(self.conv1(x)))
        x = self.conv2(x)
        return self.act_fnc(self.bn2(x + skip))


class ResUp(nn.Module):
    """
    Residual up sampling block for the decoder
    """

    def __init__(self, channel_in, channel_out, kernel_size=3, scale_factor=2):
        super(ResUp, self).__init__()

        self.conv1 = nn.Conv2d(channel_in, channel_in // 2, kernel_size, 1, kernel_size // 2)
        self.bn1 = nn.BatchNorm2d(channel_in // 2, eps=1e-4)
        self.conv2 = nn.Conv2d(channel_in // 2, channel_out, kernel_size, 1, kernel_size // 2)
        self.bn2 = nn.BatchNorm2d(channel_out, eps=1e-4)

        self.conv3 = nn.Conv2d(channel_in, channel_out, kernel_size, 1, kernel_size // 2)

        self.up_nn = nn.Upsample(scale_factor=scale_factor, mode="nearest")

        self.act_fnc = nn.ELU()

    def forward(self, x):
        x = self.up_nn(x)
        skip = self.conv3(x)
        x = self.act_fnc(self.bn1(self.conv1(x)))
        x = self.conv2(x)

        return self.act_fnc(self.bn2(x + skip))


In [16]:

# Define the encoder network
class ElevationEncoder(nn.Module):
    def __init__(self, latent_dim):
        super(ElevationEncoder, self).__init__()
        self.latent_dim = latent_dim
        self.conv_in = nn.Conv2d(1, 4, 7, 1, 3)
        self.res_down_block1 = ResDown(4, 8)
        self.res_down_block2 = ResDown(8, 16)
        self.res_down_block3 = ResDown(16,32)
        self.res_down_block4 = ResDown(32, 64)
        self.conv_mu = nn.Conv2d(64, latent_dim, 5, 1)
        self.conv_log_var = nn.Conv2d(64, latent_dim, 5, 1)
        self.act_fnc = nn.ELU()


    def forward(self, x):
        x = self.act_fnc(self.conv_in(x))
        x = self.res_down_block1(x)  # 32
        x = self.res_down_block2(x)  # 16
        x = self.res_down_block3(x)  # 8
        x = self.res_down_block4(x)
        mu = self.conv_mu(x)  # 1
        logvar = self.conv_log_var(x)  # 1

        return mu, logvar

# Define the decoder network
class ElevationDecoder(nn.Module):
    def __init__(self, latent_dim):
        super(ElevationDecoder, self).__init__()
        self.latent_dim = latent_dim
        self.conv_t_up = nn.ConvTranspose2d(latent_dim, 128, 4, 1)
        self.res_up_block1 = ResUp(128, 64)
        self.res_up_block2 = ResUp(64, 32)
        self.res_up_block3 = ResUp(32,16)
        self.res_up_block4 = ResUp(16,8)
        self.res_up_block5 = ResUp(8,7)

        self.res_down_block1 = ResDown(7,16)


        self.conv_out1 = nn.Conv2d(16, 1, 6, stride=1, padding=3)
        # self.conv_out3 = nn.Conv2d(2, 1, 3, 1, 1)

        self.act_fnc = nn.ELU()
        self.act_fnc2 = nn.ELU()


    def forward(self, x):
        x = x.view(x.shape[0], self.latent_dim, 1, 1)
        x = self.act_fnc(self.conv_t_up(x))  # 4
        x = self.res_up_block1(x)  # 8
        x = self.res_up_block2(x)  # 16
        x = self.res_up_block3(x)  # 32
        x = self.res_up_block4(x)  # 32
        x = self.res_up_block5(x)  # 32
        x = self.res_down_block1(x)  # 32

        x = self.conv_out1(x)
        return x

# Combine the encoder and decoder to form the VAE
class ElevationVAE(nn.Module):
    def __init__(self, latent_dim):
        super(ElevationVAE, self).__init__()
        self.latent_dim = latent_dim
        self.encoder = ElevationEncoder(latent_dim)
        self.decoder = ElevationDecoder(latent_dim)

    def encode(self, x):
        return self.encoder(x)

    def decode(self, z):
        return self.decoder(z)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        z = mu + eps * std
        return z

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        reconstructed_x = self.decode(z)
        return reconstructed_x, mu, logvar


# Instantiate the VAE with the desired latent_dim
latent_dim = 25
vae = ElevationVAE(latent_dim)

# Pass the input batch through the VAE
reconstructed_batch, mu, logvar = vae(first_batch)

# Check the output shape
print("Reconstructed batch shape:", reconstructed_batch.shape)


Reconstructed batch shape: torch.Size([32, 1, 65, 65])


In [17]:

# Instantiate the VAE with the desired latent_dim
latent_dim = 25
vae = ElevationVAE(latent_dim)


In [18]:
# Pass the input batch through the VAE
reconstructed_batch, mu, logvar = vae(first_batch)

# Check the output shape
print("Reconstructed batch shape:", reconstructed_batch.shape)


Reconstructed batch shape: torch.Size([32, 1, 65, 65])


In [19]:
count_parameters(vae)

443491

# Part4 Training the VAE

In [20]:
loss_weight = len(sampleCoordinatesElevationIDArrayPositionDf)/batch_sizeElevation
loss_weight

30666.65625

In [21]:
import torch.nn as nn

In [22]:
%%capture
!pip install torchmetrics

In [23]:
from torchmetrics.image.lpip import LearnedPerceptualImagePatchSimilarity

In [24]:
lpips = LearnedPerceptualImagePatchSimilarity(net_type='squeeze')

Downloading: "https://download.pytorch.org/models/squeezenet1_1-b8a52dc0.pth" to /root/.cache/torch/hub/checkpoints/squeezenet1_1-b8a52dc0.pth
100%|██████████| 4.73M/4.73M [00:00<00:00, 62.0MB/s]
/usr/local/lib/python3.10/dist-packages/torchmetrics/functional/image/lpips.py:325: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for 

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [26]:
def normalize_tensor01(tensor):
    max_val = torch.max(tensor)
    min_val = torch.min(tensor)

    # Avoid division by zero
    if max_val - min_val != 0:
        normalized_tensor = (tensor - min_val) / (max_val - min_val)
    else:
        normalized_tensor = tensor - min_val

    return normalized_tensor


def normalized_mse(tensor1, tensor2):
    # Normalize the tensors
    norm_tensor1 = normalize_tensor01(tensor1)
    norm_tensor2 = normalize_tensor01(tensor2)

    # Compute MSE
    mse_loss_normalized = F.mse_loss(norm_tensor1, norm_tensor2)
    mse_loss =  F.mse_loss(tensor1, tensor2)

    return mse_loss_normalized, mse_loss

In [27]:

class ElevationVAELoss(nn.Module):
    def __init__(self):
        super(ElevationVAELoss, self).__init__()


    def forward(self, reconstructed_x, x_final, x, mu, logvar,lpips):
        # Repeat the last two dimensions three times
        reconstructed_x_repeated =  normalize_tensor01(x_final).repeat(1, 3, 1, 1)
        x_repeated = normalize_tensor01(x).repeat(1, 3, 1, 1)

        # Compute the Mean Squared Error (MSE) reconstruction loss
        lpips = lpips(reconstructed_x_repeated, x_repeated)
        # Create the L1 loss function
        loss_value_l1 = nn.MSELoss(reduction='mean')(reconstructed_x,x)
        # Compute the KL divergence term
        kl_divergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        # Return the sum of the reconstruction loss and KL divergence term
        return kl_divergence +  (loss_value_l1)*loss_weight + lpips*loss_weight*100

In [28]:
import torch.nn.init as init

def reset_parameters(model):
    for module in model.modules():
        if isinstance(module, (nn.Conv2d, nn.Linear)):
            # Reset the weights and biases of Conv2d and Linear layers
            module.reset_parameters()
        elif isinstance(module, nn.BatchNorm2d):
            # Reset the running statistics of BatchNorm2d layers
            module.reset_running_stats()

# Assuming you have already instantiated the VAE
vae = ElevationVAE(latent_dim)

# Reinitialize the VAE's parameters
reset_parameters(vae)


In [29]:
learning_rateElevation = 0.0005

In [30]:
latent_dimElevation = 25


In [31]:

# Define the function to train the VAE
def train_vae(vae, dataloader, num_epochs, learning_rate):
    # Set the model to training mode
    vae.train()

    # Define the Mean Squared Error (MSE) loss function
    criterion = ElevationVAELoss()
    mse = torch.nn.MSELoss()

    # Define the optimizer (you can experiment with different optimizers)
    optimizer = optim.Adam(vae.parameters(), lr=learning_rate)
    total_batches = len(dataloader)
    batches_done = 0
    lpips = LearnedPerceptualImagePatchSimilarity(net_type='squeeze').to(device)

    for epoch in range(num_epochs):
        total_loss = 0.0
        batches_done =0
        mse_loss = 0
        for batch_idx, data in enumerate(dataloader):
            # Get the batch of data and move it to the device (e.g., GPU if available)
            # inputs = data
            dimensions = data.shape

            inputs = data.to(device)

            # Zero the gradients
            optimizer.zero_grad()
            # Forward pass
            reconstructed_batch, mu, logvar = vae(inputs)
            # Compute the MSE loss
            loss = criterion(reconstructed_batch, reconstructed_batch, inputs,mu, logvar,lpips)
            # Backward pass
            loss.backward()


            # Update the parameters
            optimizer.step()

            # Update the total loss for the epoch
            total_loss += loss.item()

            # Update the number of batches processed
            batches_done += 1
            mse_loss += mse(normalize_tensor01(reconstructed_batch),normalize_tensor01(inputs))

            # Print the progress when a tenth of the epoch is completed
            if batches_done % (len(dataloader) // 10) == 0:
                print(f"Epoch [{epoch+1}/{num_epochs}] - Progress: {batches_done}/{len(dataloader)} - Total Loss: {total_loss / (len(dataloader) // 10)}, {mse_loss.item() / (len(dataloader) // 10)}")
                total_loss = 0
                mse_loss = 0

        # Print the average loss for the epoch
        print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {total_loss / ((len(dataloader) - len(dataloader) //10 * 9 ) % 10)}")


In [32]:

# Example usage:'device = torch.device("cuda" if torch.cuda.is_available() else "cpu")'
# Assuming you have the training data in 'train_dataloader' and a device set, e.g.,
# Instantiate the VAE with the desired latent_dim
vae = ElevationVAE(latent_dimElevation).to(device)

# Define the number of epochs and learning rate
num_epochs = 3


In [33]:


# Compute average normalized MSE for the first 100 batches
def compute_average_mse(dataloader):
    mse_values = []
    count = 0

    for data in dataloader:
        if count >= 10000:
            break

        # Generate random "predictions" just for the sake of the example
        # In a real scenario, these would be the model's output
        reconstructed_batch, _, _ = vae(data.to(device))

        mse,mse2 = normalized_mse(data.cpu(), reconstructed_batch.cpu())
        mse_values.append(mse.item())

        count += 1

    average_mse = sum(mse_values) / len(mse_values)
    print("Average Normalized MSE:", average_mse)



In [ ]:
# Train the VAE
train_vae(vae, dataLoaderElevation, num_epochs, learning_rateElevation )

<ipython-input-6-59023ef0d4a7>:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  fullArray = torch.load(self.file_path+ID+self.file_extension)
<ipython-input-6-59023ef0d4a7>

Epoch [1/3] - Progress: 3066/30667 - Total Loss: 1825419919.6060014, 0.0060603234477083585
Epoch [1/3] - Progress: 6132/30667 - Total Loss: 277564664.4174821, 0.0017209542940740715
Epoch [1/3] - Progress: 9198/30667 - Total Loss: 170139612.14220482, 0.0017131514900740035
Epoch [1/3] - Progress: 12264/30667 - Total Loss: 136505340.50815395, 0.0020688852296494534
Epoch [1/3] - Progress: 15330/30667 - Total Loss: 116598778.03587736, 0.00213207688428029
Epoch [1/3] - Progress: 18396/30667 - Total Loss: 104738170.07436399, 0.0020545667398232537
Epoch [1/3] - Progress: 21462/30667 - Total Loss: 101816615.62622309, 0.001944859978115333
Epoch [1/3] - Progress: 24528/30667 - Total Loss: 112163035.96151337, 0.0019479292479513209
Epoch [1/3] - Progress: 27594/30667 - Total Loss: 86793130.74624918, 0.0017371546265663698


In [ ]:
torch.save(vae, 'vaeElevation3Epoch.pt')

In [ ]:
!mv vaeElevation3Epoch.pt "/content/drive/MyDrive/Colab Notebooks/"

In [ ]:
# Example usage
# Replace `your_dataloader` with the DataLoader you have
compute_average_mse(dataLoaderElevation)

In [ ]:
train_vae(vae, dataLoaderElevation, 2, learning_rateElevation )

In [ ]:
torch.save(vae, 'vaeElevation5Epoch.pt')

In [ ]:
!mv vaeElevation5Epoch.pt "/content/drive/MyDrive/Colab Notebooks/"

In [ ]:
# Example usage
# Replace `your_dataloader` with the DataLoader you have
compute_average_mse(dataLoaderElevation)

In [ ]:
#train_vae(vae, dataLoaderElevation, 2, learning_rateElevation )

In [ ]:
count_parameters(vae)

# Part 5. Plotting the results ofthe Elevation VAE

In [ ]:
def get_random_batch(dataloader):
    # Get the total number of batches in the DataLoader
    num_batches = len(dataloader)

    # Generate a random index to select a batch
    random_batch_index = torch.randint(0, 100, (1,))

    # Iterate through the DataLoader to find the batch at the random index
    for i, batch in enumerate(dataloader):
        if i == random_batch_index:
            return batch


In [ ]:
randomBatch = get_random_batch(dataLoaderElevation)

In [ ]:
import torch
import matplotlib.pyplot as plt

def compare_vae_reconstruction(vae, data_batch, device):
    # Set the VAE to evaluation mode
    vae.eval()

    # Get the batch size and number of channels
    batch_size, num_channels, height, width = data_batch.size()

    # Get the reconstructed images from the VAE
    with torch.no_grad():
        dimensions = data_batch.shape
        data_batch = data_batch
        print(dimensions)

        reconstructed_batch, _, _ = vae(data_batch.to(device))
    # Convert the tensors to numpy arrays and transpose the dimensions
    original_images = data_batch.cpu().numpy().transpose(0,2,3,1)
    print(reconstructed_batch.shape)
    reconstructed_images = reconstructed_batch.cpu().numpy().transpose(0,2,3,1)
    print(reconstructed_images.shape)


    # Plot the original and reconstructed images side by side
    plt.figure(figsize=(100, 100))
    for i in range(batch_size):
        plt.subplot(batch_size, 16, i*16 + 1)
        plt.imshow(original_images[i])
        plt.axis('off')
        plt.title('Original')

        plt.subplot(batch_size, 16, i*16 + 2)
        plt.imshow(reconstructed_images[i])
        plt.axis('off')
        plt.title('Reconstructed')


    plt.tight_layout()
    plt.show()

# Usage
# Assuming 'vae' is your trained Variational Autoencoder model
# and 'data_batch' is your batch of input images
# 'device' should be the device on which your model is (e.g., 'cuda' or 'cpu')
compare_vae_reconstruction(vae, randomBatch, device)
